In [102]:
import sys
sys.path.append('../')
from utilities.task_service import get_authentication_token, get_task, send_answer

from auth import apikey
import requests
import json
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

from dotenv import load_dotenv
load_dotenv(override=True)

task_name = "people"

In [9]:
# pobranie linków z urla
url = f"https://tasks.aidevs.pl/data/people.json"

try:
    response_data = requests.post(url)
    response_data.raise_for_status()  # Sprawdzenie, czy odpowiedź jest sukcesem (status kod 2xx)
    data = response_data.json()
except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")

with open("data.json", "w", encoding='utf8') as file:
    json.dump(data, file, indent=1)

In [120]:
# stwrzenie dataframe'a z danymi
df = pd.DataFrame(data)
df

,imie,nazwisko,wiek,o_mnie,ulubiona_postac_z_kapitana_bomby,ulubiony_serial,ulubiony_film,ulubiony_kolor
0,Dariusz,Kaczor,46,niekiedy lubie jeść lody. Mieszkam w Radomiu. ...,Admirał Gwiezdnej Floty,Stranger Things,Avengers,morski
1,Katarzyna,Rumcajs,32,lubie zjadać lody. Mieszkam w Łodzi. Interesuj...,nie oglądam,Big Bang Theory,The Lord of the Rings,magenta
2,Renata,Pizza,49,czasami lubie zjadać pizzę. Mieszkam w Łodzi. ...,nie pamiętam tych wsystkich imion,Walking Dead,The Prestige,czerwony
3,Katarzyna,Kot,61,niekiedy lubie jeść pizzę. Mieszkam w Krakowie...,nie pamiętam tych wsystkich imion,Game of Thrones,Titanic,oliwkowy
4,Władysław,Bajorko,24,niekiedy lubie jeść pizzę. Mieszkam w Poznaniu...,nie oglądam,LOST,The Sixth Sense,malinowy
...,...,...,...,...,...,...,...,...
1382,Agata,Kaczka,41,niekiedy lubie spożywać pizzę. Mieszkam w Chrz...,Chorąży Torpeda,Dom z papieru,Titanic,oliwkowy
1383,Jan,Kot,42,czasami lubie jeść lody. Mieszkam w Radomiu. I...,Sułtan Kosmitów,Big Bang Theory,The Lion King,złoty
1384,Rafał,Banan,36,czasami lubie jeść lody. Mieszkam w Łodzi. Int...,nie oglądam,Moda na sukces,The Departed,biały
1385,Maciej,Kaczor,62,niekiedy lubie zjadać lody. Mieszkam w Radomiu...,Admirał Gwiezdnej Floty,Dom z papieru,Titanic,koralowy


In [121]:
# autoryzacja

token = get_authentication_token(apikey, task_name)
if token:
    print("Otrzymano token.")
else:
    print("Nie udało się uzyskać tokenu.")

Otrzymano token.


In [122]:
# pobranie danych do zadania

response = get_task(token)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania zapytania.")

Odpowiedź z serwera: {'code': 0, 'msg': 'retrieve the data set (JSON) and answer the question. The question will change every time the task is called. I only ask about favourite colour, favourite food and place of residence', 'data': 'https://tasks.aidevs.pl/data/people.json', 'question': 'Gdzie mieszka Krysia Ludek?', 'hint1': 'Does everything have to be handled by the language model?', 'hint2': 'prepare knowledge DB for this task'}


Informacja o ulubionym jedzeniu jest ukryta w pierwszym zdaniu informacji "o mnie",
 natomiast informacja o miejscu zamieszkania w drugim zdaniu:

In [123]:
# jedzenie
print(df['o_mnie'][0].split(".")[0])

# miejsce zamieszkania
print(df['o_mnie'][156].split(". ")[1])

niekiedy lubie jeść lody
Mieszkam w Poznaniu


Możemy wyodrębnić te informacje i zapisać je w nowych kolumnach dataframe'a:

In [124]:
df['ulubione_jedzenie'] = df['o_mnie'].apply(lambda x: x.split('.')[0])
df['miejsce_zamieszkania'] = df['o_mnie'].apply(lambda x: x.split('. ')[1])
df.head()

,imie,nazwisko,wiek,o_mnie,ulubiona_postac_z_kapitana_bomby,ulubiony_serial,ulubiony_film,ulubiony_kolor,ulubione_jedzenie,miejsce_zamieszkania
0,Dariusz,Kaczor,46,niekiedy lubie jeść lody. Mieszkam w Radomiu. ...,Admirał Gwiezdnej Floty,Stranger Things,Avengers,morski,niekiedy lubie jeść lody,Mieszkam w Radomiu
1,Katarzyna,Rumcajs,32,lubie zjadać lody. Mieszkam w Łodzi. Interesuj...,nie oglądam,Big Bang Theory,The Lord of the Rings,magenta,lubie zjadać lody,Mieszkam w Łodzi
2,Renata,Pizza,49,czasami lubie zjadać pizzę. Mieszkam w Łodzi. ...,nie pamiętam tych wsystkich imion,Walking Dead,The Prestige,czerwony,czasami lubie zjadać pizzę,Mieszkam w Łodzi
3,Katarzyna,Kot,61,niekiedy lubie jeść pizzę. Mieszkam w Krakowie...,nie pamiętam tych wsystkich imion,Game of Thrones,Titanic,oliwkowy,niekiedy lubie jeść pizzę,Mieszkam w Krakowie
4,Władysław,Bajorko,24,niekiedy lubie jeść pizzę. Mieszkam w Poznaniu...,nie oglądam,LOST,The Sixth Sense,malinowy,niekiedy lubie jeść pizzę,Mieszkam w Poznaniu


In [125]:
# znalezienie tematyki pytania
chat = ChatOpenAI(
    model='gpt-4'
)

# znalezienie imienia występującego w pytaniu
question_info = chat.invoke([
    SystemMessage(f"""Znajdź imię (w formie oficjalnej), nazwisko oraz tematykę pytania (ulubiony_kolor, miejsce_zamieszkania lub ulubione_jedzenie). Odpowiedź zwróć w formie JSON. Na przykład:
                  ###
                  user: jaki kolor się podoba Mariuszowi Kaczorowi?
                  AI: ["Mariusz", "Kaczor", "ulubiony_kolor"]

                  user: gdzie mieszka Ania Czołg?
                  AI: ["Anna", "Czołg", "miejsce_zamieszkania"]

                  user: co lubi jeść Kasia Duch?
                  AI: ["Katarzyna", "Duch", "ulubione_jedzenie"]
                  ###
                  """),
    HumanMessage(f"Pytanie: {response['question']}")
])
print(question_info.content)

["Krystyna", "Ludek", "miejsce_zamieszkania"]


In [126]:
# wyciągnięcie z odpowiedzi modelu konkretnych danych
json_question_info = json.loads(question_info.content)

imie = json_question_info[0]
nazwisko = json_question_info[1]
tematyka = json_question_info[2]

# wyfiltrowanie osoby i szukanej odpowiedzi
df_szukana_osoba = df[(df['imie'] == imie) & (df['nazwisko'] == nazwisko)]
answer = df_szukana_osoba[tematyka].iloc[0]

In [129]:
print(answer)

Mieszkam w Warszawie


In [128]:
# wysłanie odpowiedzi

response = send_answer(token, answer)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania odpowiedzi.")

Odpowiedź z serwera: {'code': 0, 'msg': 'OK', 'note': 'CORRECT'}
